In [ ]:
from astropy.cosmology import Planck18 as cosmo
from astropy.io import fits
import astropy.units as u

import numpy as np

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})
import matplotlib.pyplot as plt

from jpasLAEs.utils import flux_to_mag, smooth_hist

In [ ]:
cat = fits.open('../../catalogs/COLA1_O3doublet_catalog_1.3.fits')[1].data
cat.columns

In [ ]:
cat_eiger = fits.open('UV_slopes_EIGER_field.fits')[1].data
cat_eiger.columns

In [ ]:
def continuum_powerlaw(wavelength, norm, beta, redshift):
    return norm*(wavelength/(1500.*(1+redshift)))**beta

In [ ]:
def app_m_to_flux_nu(m):
    """
    :param m: apparent magnitude
    :return: the (observed) flux density
    """
    return 10 ** (-(m + 48.60) / 2.5)


def flux_to_lum(flux, z):
    """
    :param flux: flux
    :param z: redshift
    :return: luminosity
    """
    return flux * 4 * np.pi * cosmo.luminosity_distance(z).to("cm").value ** 2

def flux_nu_to_lum_nu(flux_nu, z):
    """
    :param flux_nu: the (observed) flux density
    :param z: redshift
    :return: the (intrinsic) luminosity density
    """
    return flux_to_lum(flux_nu, z) / (1 + z)

In [ ]:
this_redshift = 6.5916
this_obs_1500 = 1500 * (1 + this_redshift)

args = (this_obs_1500, 4.18, -2.38, this_redshift)
this_f_UV = continuum_powerlaw(*args) * 1e-20

this_distmod = cosmo.distmod(this_redshift).value

this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
this_m_UV[~np.isfinite(this_m_UV)] = -99

this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)
this_M_UV

In [ ]:
N_iter = 1000

N_sources = len(cat)

M_UV = np.empty(N_sources)
M_UV_err_up = np.empty(N_sources)
M_UV_err_down = np.empty(N_sources)

L_UV = np.empty(N_sources)
L_UV_err_up = np.empty(N_sources)
L_UV_err_down = np.empty(N_sources)

cHb = 4.79e-13

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat['beta_powerlaw'][src], cat['beta_powerlaw_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm_powerlaw'][src], cat['norm_powerlaw_err'][src], size=N_iter)
    this_rand_Hb = np.random.normal(cat['f_Hb'][src], cat['f_Hb_err'][src], size=N_iter)

    this_redshift = cat['z_O3doublet_combined_n'][src]

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift) * 0.47 / 0.4

    this_lum_dist = cosmo.luminosity_distance(this_redshift).to(u.cm).value

    this_L_Hb = this_rand_Hb * 1e-18 * this_lum_dist**2 * 4*np.pi

    this_xi_ion = np.log10(this_L_Hb / this_L_UV / cHb)

    this_M_UV_percentiles = np.percentile(this_M_UV,[16, 50, 84])
    this_f_UV_percentiles = np.percentile(this_f_UV, [16, 50, 84])
    this_xi_ion_percentiles = np.percentile(this_xi_ion, [16, 50, 84])

    M_UV[src] = this_M_UV_percentiles[1]
    M_UV_err_up[src] = this_M_UV_percentiles[2] - M_UV[src]
    M_UV_err_down[src] = M_UV[src] - this_M_UV_percentiles[0]

    L_UV[src] = this_f_UV_percentiles[1]
    L_UV_err_up[src] = this_f_UV_percentiles[2] - L_UV[src]
    L_UV_err_down[src] = L_UV[src] - this_f_UV_percentiles[0]


In [ ]:
from jpasLAEs.utils import flux_to_mag
import sys
sys.path.insert(0, '..')
from dust_correction import SFR_XION_dust_corrected

# Compute SFR
# For now we assume the same value of xi_ion as in Matthee2023.
SFR_C_UV_COLA1 = 43.51
SFR_C_Hb_COLA1 = 41.78

SFR_C_UV = 43.36
SFR_C_Hb = 41.59

xi_ion = np.empty(N_sources)
xi_ion_err_up = np.empty(N_sources)
xi_ion_err_down = np.empty(N_sources)


for src in range(len(cat)):
    # if cat['NUMBER'][src] == 9269:
    #     this_SFR_C_Hb = SFR_C_Hb_COLA1
    #     this_SFR_C_UV = SFR_C_UV_COLA1
    #     this_Hg = 2.11797,
    #     this_Hg_err = 0.20597
    #     this_Hb = 4.10216
    #     this_Hb_err = 0.16479
    # else:
    this_balmer_dec = 0.47 / 0.4
    this_SFR_C_Hb = SFR_C_Hb
    this_SFR_C_UV = SFR_C_UV
    this_Hb = cat['f_Hb'][src]
    this_Hb_err = cat['f_Hb_err'][src]
    this_Hg = 0.4 * this_Hb
    this_Hg_err = 0


    this_redshift = cat['z_O3doublet_combined_n'][src]
    N_iter = 5000
    this_rand_beta = np.random.normal(cat['beta_powerlaw'][src], cat['beta_powerlaw_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm_powerlaw'][src], cat['norm_powerlaw_err'][src], size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    _, _, xi_ion_percs = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                 this_Hb, this_Hb_err,
                                                 this_M_UV, this_M_UV_err,
                                                 this_redshift, this_SFR_C_Hb,
                                                 this_SFR_C_UV)

    xi_ion[src] = np.nanmedian(xi_ion_percs)
    xi_ion_err_up[src] = np.nanpercentile(xi_ion_percs, 84) - xi_ion[src]
    xi_ion_err_down[src] = xi_ion[src] - np.nanpercentile(xi_ion_percs, 16)

In [ ]:
# Compute xi_ion with the dust correction of COLA1
src = np.where(cat['NUMBER'] == 9269)[0][0]
for src in range(len(cat)):
    if cat['NUMBER'][src] == 9269:
        this_SFR_C_Hb = SFR_C_Hb_COLA1
        this_SFR_C_UV = SFR_C_UV_COLA1
        this_Hg = 2.11797,
        this_Hg_err = 0.20597
        this_Hb = 4.10216
        this_Hb_err = 0.16479
    else:
        continue

    this_redshift = cat['z_O3doublet_combined_n'][src]
    N_iter = 5000
    this_rand_beta = np.random.normal(cat['beta_powerlaw'][src], cat['beta_powerlaw_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm_powerlaw'][src], cat['norm_powerlaw_err'][src], size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    _, _, xi_ion_COLA1_percs = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                 this_Hb, this_Hb_err,
                                                 this_M_UV, this_M_UV_err,
                                                 this_redshift, this_SFR_C_Hb,
                                                 this_SFR_C_UV)

In [ ]:
muvperc = np.percentile(this_M_UV, [16, 50, 84])
print(muvperc[1], muvperc[1] - muvperc[[True, False, True]])

In [ ]:
N_iter = 1000

N_sources_eiger = len(cat_eiger)

M_UV_eiger = np.empty(N_sources_eiger)
M_UV_err_up_eiger = np.empty(N_sources_eiger)
M_UV_err_down_eiger = np.empty(N_sources_eiger)

L_UV_eiger = np.empty(N_sources_eiger)
L_UV_err_up_eiger = np.empty(N_sources_eiger)
L_UV_err_down_eiger = np.empty(N_sources_eiger)

for src in range(N_sources_eiger):
    this_rand_beta = np.random.normal(cat_eiger['beta'][src], cat_eiger['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat_eiger['norm'][src], cat_eiger['norm_err'][src], size=N_iter)
    this_rand_Hb = np.random.normal(cat_eiger['f_Hb'][src], cat_eiger['f_Hb_err'][src], size=N_iter)

    this_redshift = cat_eiger['z_O3doublet_combined_n'][src]

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift) * 0.47 / 0.4

    this_lum_dist = cosmo.luminosity_distance(this_redshift).to(u.cm).value

    this_L_Hb = this_rand_Hb * 1e-18 * this_lum_dist**2 * 4*np.pi

    this_M_UV_percentiles = np.percentile(this_M_UV,[16, 50, 84])
    this_f_UV_percentiles = np.percentile(this_f_UV, [16, 50, 84])
    this_xi_ion_percentiles = np.percentile(this_xi_ion, [16, 50, 84])

    M_UV_eiger[src] = this_M_UV_percentiles[1]
    M_UV_err_up_eiger[src] = this_M_UV_percentiles[2] - M_UV_eiger[src]
    M_UV_err_down_eiger[src] = M_UV_eiger[src] - this_M_UV_percentiles[0]

    L_UV_eiger[src] = this_f_UV_percentiles[1]
    L_UV_err_up_eiger[src] = this_f_UV_percentiles[2] - L_UV_eiger[src]
    L_UV_err_down_eiger[src] = L_UV_eiger[src] - this_f_UV_percentiles[0]

In [ ]:
# Compute SFR
# For now we assume the same value of xi_ion as in Matthee2023.
xi_ion_eiger = np.empty(N_sources_eiger)
xi_ion_err_up_eiger = np.empty(N_sources_eiger)
xi_ion_err_down_eiger = np.empty(N_sources_eiger)


for src in range(len(cat_eiger)):
    this_balmer_dec = 0.47 / 0.4
    this_SFR_C_Hb = SFR_C_Hb
    this_SFR_C_UV = SFR_C_UV
    this_Hb = cat_eiger['f_Hb'][src]
    this_Hb_err = cat_eiger['f_Hb_err'][src]
    this_Hg = 0.4 * this_Hb
    this_Hg_err = 0


    this_redshift = cat_eiger['z_O3doublet_combined_n'][src]
    N_iter = 5000
    this_rand_beta = np.random.normal(cat_eiger['beta'][src], cat_eiger['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat_eiger['norm'][src], cat_eiger['norm_err'][src], size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    _, _, xi_ion_percs = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                 this_Hb, this_Hb_err,
                                                 this_M_UV, this_M_UV_err,
                                                 this_redshift, this_SFR_C_Hb,
                                                 this_SFR_C_UV)

    xi_ion_eiger[src] = np.nanmedian(xi_ion_percs)
    xi_ion_err_up_eiger[src] = np.nanpercentile(xi_ion_percs, 84) - xi_ion_eiger[src]
    xi_ion_err_down_eiger[src] = xi_ion_eiger[src] - np.nanpercentile(xi_ion_percs, 16)

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(cat['F115W_AUTO_mag'], M_UV, yerr=[M_UV_err_up, M_UV_err_down], ls='', fmt='o')

mask_COLA1 = cat['NUMBER'] == 9269
print(M_UV[mask_COLA1][0])
ax.errorbar(cat['F115W_AUTO_mag'][mask_COLA1], M_UV[mask_COLA1],
             yerr=[M_UV_err_up[mask_COLA1], M_UV_err_down[mask_COLA1]], ls='', fmt='*', ms=20, c='r')

ax.set_ylim(-22, -16)
ax.set_xlim(24, 30)

ax.set_ylabel(r'$M_{\rm UV}$')
ax.set_xlabel('F115W [magAB]')

plt.show()

In [ ]:
def M_UV_bin_percentiles(values, xx, xx_min, xx_max, bin_width, step):
    if xx_max <= xx_min:
        raise ValueError('value_max has to be greater than value_min')
    if len(values) != len(xx):
        raise Exception('Lengths of values and xx arrays must be the same.')

    centers = np.arange(xx_min + step * 0.5, xx_max, step)
    N_steps = len(centers)
    perc_16 = np.zeros(N_steps, dtype=float)
    perc_50 = np.zeros(N_steps, dtype=float)
    perc_84 = np.zeros(N_steps, dtype=float)

    for j in range(N_steps):
        this_mask = (
            (xx >= centers[j] - bin_width * 0.5)
            & (xx < centers[j] + bin_width * 0.5)
        )

        perc_16[j] = np.nanpercentile(values[this_mask], 16)
        perc_50[j] = np.nanpercentile(values[this_mask], 50)
        perc_84[j] = np.nanpercentile(values[this_mask], 84)

    return [perc_16, perc_50, perc_84], centers

In [ ]:
f_O3_Hbeta = cat['f_O3_5008'] + cat['f_O3_4960'] + cat['f_Hb']
f_O3_Hbeta_err = (cat['f_O3_5008_err']**2 + cat['f_O3_4960']**2 + cat['f_Hb']**2) ** 0.5

EW = np.empty(N_sources)
EW_err_up = np.empty(N_sources)
EW_err_down = np.empty(N_sources)

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat['beta_powerlaw'][src], cat['beta_powerlaw_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm_powerlaw'][src], cat['norm_powerlaw_err'][src], size=N_iter)

    this_redshift = cat['z_O3doublet_combined_n'][src]

    this_obs_O3_lambda = 5008.24 * (1 + this_redshift)

    args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
    
    # Assume Hg/Hb = 0.4

    this_O3_cont = continuum_powerlaw(*args)

    this_O3_Hbeta_f = np.random.normal(f_O3_Hbeta[src], f_O3_Hbeta_err[src], size=N_iter)

    this_EW_percs = np.percentile(this_O3_Hbeta_f / this_O3_cont, [16, 50, 84]) / (1 + this_redshift) * 100

    EW[src] = this_EW_percs[1]
    EW_err_up[src] = this_EW_percs[2] - this_EW_percs[1]
    EW_err_down[src] = this_EW_percs[1] - this_EW_percs[0]

In [ ]:
src = np.where(cat['NUMBER'] == 9269)[0][0]

this_rand_beta = np.random.normal(cat['beta_powerlaw'][src], cat['beta_powerlaw_err'][src], size=N_iter)
this_rand_norm = np.random.normal(cat['norm_powerlaw'][src], cat['norm_powerlaw_err'][src], size=N_iter)
this_rand_HgHb = np.random.normal(0.516, 0.054, size=N_iter)

this_redshift = cat['z_O3doublet_combined_n'][src]

this_obs_O3_lambda = 5008.24 * (1 + this_redshift)
this_obs_Hb_lambda = 4862.69 * (1 + this_redshift)

# Assume Hg/Hb = 0.4
args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
this_O3_cont = continuum_powerlaw(*args) * 1e-20

args = (this_obs_Hb_lambda, this_rand_norm, this_rand_beta, this_redshift)
this_Hb_cont = continuum_powerlaw(*args) * 1e-20

this_O3_Hbeta_f = np.random.normal(f_O3_Hbeta[src], f_O3_Hbeta_err[src], size=N_iter)
this_Hb_f = np.random.normal(cat['f_Hb'][src], cat['f_Hb_err'][src], size=N_iter)

EW_COLA1_percs = np.percentile(this_O3_Hbeta_f * 1e-18 / this_O3_cont, [16, 50, 84]) / (1 + this_redshift)
EW_Hb_COLA1_percs = np.percentile(this_Hb_f * 1e-18 / this_Hb_cont, [16, 50, 84]) / (1 + this_redshift)
print(EW_COLA1_percs)

In [ ]:
import astropy.units as u
import astropy.constants as c
# this_rand_beta = np.random.normal(-3.24, 0.45, size=N_iter)
# this_rand_norm = np.random.normal(4.58, 0.38, size=N_iter)
args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
this_O3_cont = continuum_powerlaw(*args) * 1e-20
(np.percentile(this_O3_cont, [16, 50, 84]) * u.erg * u.cm**-2 * u.s**-1 * u.AA**-1 * (0.5008 * 7.6 * u.micron)**2 / c.c).to(u.nJy)

In [ ]:
f_O3_Hbeta_eiger = cat_eiger['f_O3_5008'] + cat_eiger['f_O3_4960'] + cat_eiger['f_Hb']
f_O3_Hbeta_err_eiger = (cat_eiger['f_O3_5008_err']**2 + cat_eiger['f_O3_4960']**2 + cat_eiger['f_Hb']**2) ** 0.5

EW_eiger = np.empty(N_sources_eiger)
EW_err_up_eiger = np.empty(N_sources_eiger)
EW_err_down_eiger = np.empty(N_sources_eiger)

for src in range(N_sources_eiger):
    this_rand_beta = np.random.normal(cat_eiger['beta'][src], cat_eiger['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat_eiger['norm'][src], cat_eiger['norm_err'][src], size=N_iter)

    this_redshift = cat_eiger['z_O3doublet_combined_n'][src]

    this_obs_O3_lambda = 5008.24 * (1 + this_redshift)

    args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
    
    # Assume Hg/Hb = 0.4
    this_O3_cont = continuum_powerlaw(*args)

    this_O3_Hbeta_f = np.random.normal(f_O3_Hbeta_eiger[src], f_O3_Hbeta_err_eiger[src], size=N_iter)

    this_EW_percs = np.percentile(this_O3_Hbeta_f / this_O3_cont, [16, 50, 84]) / (1 + this_redshift) * 100

    EW_eiger[src] = this_EW_percs[1]
    EW_err_up_eiger[src] = this_EW_percs[2] - this_EW_percs[1]
    EW_err_down_eiger[src] = this_EW_percs[1] - this_EW_percs[0]

In [ ]:
# EIGER prospector
cat_eiger_prosp = fits.open('/home/alberto/cosmos/ista/COLA1/catalogs/eiger/EIGER_5fields_O3doublets_SYSTEMS_31102023_with_Prosp.fits')[1].data
cat_eiger_prosp.columns

In [ ]:
M_UV = cat['muv']
EW = (cat['EW_O3'] + cat['EW_Hb'])
EW_eiger = (cat_eiger_prosp['EWO3_prosp'] + cat_eiger_prosp['EWHb_prosp'])
M_UV_eiger = cat_eiger_prosp['MUV_prosp']
xi_ion = cat['xi_ion_hb']
# xi_ion_eiger = cat_eiger_prosp['x']

fig, axes = plt.subplots(3, figsize=(6, 14), sharex=True)

ax_beta = axes[0]
ax_ew = axes[1]
ax_xi = axes[2]

ax_beta.errorbar(M_UV, cat['beta'],
            ls='', fmt='o', c='lightsteelblue', label='All C1F', zorder=-1, rasterized=True)

ax_beta.errorbar(M_UV_eiger, cat_eiger['beta'],
            ls='', fmt='o', c='dimgray', label='All EIGER', zorder=-2, rasterized=True)

M_UV_median = np.nanmedian(M_UV)
EW_median = np.nanmedian(EW)
M_UV_median_eiger = np.nanmedian(M_UV_eiger)
EW_median_eiger = np.nanmedian(EW_eiger)

ax_beta.errorbar(M_UV_median, np.nanmedian(cat['beta']),
            yerr=np.abs(np.nanmedian(
                cat['beta']) - np.nanpercentile(cat['beta'], [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median - np.nanpercentile(M_UV, [16, 84])).reshape(2, 1),
            zorder=999, fmt='D', ms=10, capsize=5, mfc='lightsteelblue', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average C1F')

ax_beta.errorbar(M_UV_median_eiger, np.nanmedian(cat_eiger['beta']),
            yerr=np.abs(np.nanmedian(
                cat_eiger['beta']) - np.nanpercentile(cat_eiger['beta'], [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median_eiger - np.nanpercentile(M_UV_eiger, [16, 84])).reshape(2, 1),
            zorder=998, fmt='s', ms=10, capsize=5, mfc='dimgray', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average EIGER')

ax_beta.errorbar(M_UV[mask_COLA1], cat['beta'][mask_COLA1],
            yerr=cat['beta_powerlaw_err'][mask_COLA1],
            zorder=999, fmt='D', ms=10, capsize=5, mfc='r', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1')

# https://ui.adsabs.harvard.edu/search/q=%20%20author%3A%22%5EBouwens%22%20year%3A2014&sort=date%20desc%2C%20bibcode%20desc&p_=0
# bouwens 2014 793 apj
xx = np.arange(-23, -18.8, 0.01)
xx_2 = np.arange(-18.8, -16, 0.01)
yy = -2.22 + -0.24*(xx - -18.8)
yy_2 = -2.22 + -0.08*(xx_2 - -18.8)

ax_beta.plot(xx, yy, color='indigo', ls='--', lw=3,
        label=r'LBGs $z\sim6$ (Bouwens+14)')
ax_beta.plot(xx_2, yy_2, color='indigo', ls='--', lw=3)

print(f'COLA1: beta = {cat['beta'][mask_COLA1][0]:0.2} +/- {cat['beta_powerlaw_err'][mask_COLA1][0]:0.2f}')
print(f'All: beta = {np.nanmedian(cat['beta']):0.2} +/- {
      np.abs(np.nanmedian(cat['beta']) - np.nanpercentile(cat['beta'], [16, 84]))}')

### Compute median and percentiles in bins of M_UV ###
beta_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([cat['beta'], cat_eiger['beta']]),
                                                    np.concatenate([M_UV, M_UV_eiger]),
                                                    -25, -15, 1.4, 0.1)
ax_beta.plot(M_UV_bin_centers, beta_percs[1], c='k', lw=3)
ax_beta.plot(M_UV_bin_centers, beta_percs[1], c='darkcyan', lw=2,
             label='Median (C1F + EIGER)')
ax_beta.fill_between(M_UV_bin_centers, beta_percs[0], beta_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######


# ax_beta.set_xlabel(r'$M_{\rm UV}$')
ax_beta.set_ylabel(r'$\beta_{\rm UV}$')

ax_beta.legend(fontsize=10, markerfirst=False)

ax_beta.set_xlim(-22.5, -16.5)
ax_beta.set_ylim(-4.5, 2.5)

ax_beta.tick_params(direction='in', which='both')
ax_beta.xaxis.set_ticks_position('both')
ax_beta.yaxis.set_ticks_position('both')
ax_beta.invert_xaxis()


#################
#################

mask_good_EW = EW > 0

ax_ew.errorbar(M_UV[mask_good_EW], EW[mask_good_EW],
            # yerr=[EW_err_up, EW_err_down],
            ls='',
            fmt='o', c='lightsteelblue', label='All C1F', zorder=-1, rasterized=True)

ax_ew.errorbar(M_UV_eiger, EW_eiger,
            # yerr=[EW_err_up_eiger, EW_err_down_eiger],
            ls='',
            fmt='o', c='dimgray', label='All EIGER', zorder=-2, rasterized=True)

ax_ew.errorbar(M_UV[mask_COLA1][0], EW_COLA1_percs[1],
               yerr=np.abs([EW_COLA1_percs[0] - EW_COLA1_percs[1],
                               EW_COLA1_percs[2] - EW_COLA1_percs[1]]).reshape(2, 1),
               xerr=np.abs([M_UV_err_down[mask_COLA1][0], M_UV_err_up[mask_COLA1][0]]).reshape(2, 1),
               ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
               elinewidth=2, capthick=2, mew=1.5, label=r'COLA1', zorder=99999)

print(f'COLA1: {EW[mask_COLA1][0]:0.2f} +/- {EW_err_up[mask_COLA1], EW_err_down[mask_COLA1]}')

M_UV_median = np.nanmedian(M_UV[mask_good_EW])
EW_median = np.nanmedian(EW[mask_good_EW])
M_UV_median_eiger = np.nanmedian(M_UV_eiger)
EW_median_eiger = np.nanmedian(EW_eiger)

print(f'All sources: EW = {EW_median:0.2f} +/- {np.abs(EW_median - np.nanpercentile(EW, [16, 84]))}')
print(f'All EIGER: EW = {EW_median_eiger:0.2f} +/- {np.abs(EW_median_eiger - np.nanpercentile(EW_eiger, [16, 84]))}')
ax_ew.errorbar(M_UV_median, EW_median,
            yerr=np.abs(EW_median - np.nanpercentile(EW[mask_good_EW], [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median - np.nanpercentile(M_UV[mask_good_EW], [16, 84])).reshape(2, 1),
            zorder=999, fmt='D', ms=10, capsize=5, mfc='lightsteelblue', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average C1F')

ax_ew.errorbar(M_UV_median_eiger, EW_median_eiger,
            yerr=np.abs(EW_median_eiger - np.nanpercentile(EW_eiger, [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median_eiger - np.nanpercentile(M_UV_eiger, [16, 84])).reshape(2, 1),
            zorder=999, fmt='s', ms=10, capsize=5, mfc='dimgray', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average EIGER')

### Compute median and percentiles in bins of M_UV ###
ew_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([EW, EW_eiger]),
                                                    np.concatenate([M_UV, M_UV_eiger]),
                                                    -25, -15, 1, 0.1)
ax_ew.plot(M_UV_bin_centers, ew_percs[1], c='k', lw=3)
ax_ew.plot(M_UV_bin_centers, ew_percs[1], c='darkcyan', lw=2,
             label='Median (C1F + EIGER)')
ax_ew.fill_between(M_UV_bin_centers, ew_percs[0], ew_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######

ax_ew.set_ylim(50, 1e5)
ax_ew.set_xlim(-22.5, -16.5)

# ax_ew.set_xlabel(r'$M_{\rm UV}$')
ax_ew.set_ylabel(r'EW$_0$([OIII] + H$\beta$) [\AA]')

ax_ew.set_yscale('log')

ax_ew.legend(fontsize=10, markerfirst=False)

ax_ew.tick_params(direction='in', which='both')
ax_ew.xaxis.set_ticks_position('both')
ax_ew.yaxis.set_ticks_position('both')
ax_ew.invert_xaxis()

##################
##################

mask_Hb_SN = cat['f_Hb'] / cat['f_Hb_err'] > 3
ax_xi.errorbar(M_UV[mask_Hb_SN], xi_ion[mask_Hb_SN], ls='',
            fmt='o', c='lightsteelblue', label=r'S/N(H$\beta$) $>$ 3 C1F', zorder=-1, rasterized=True)

mask_Hb_SN = cat_eiger['f_Hb'] / cat_eiger['f_Hb_err'] > 3
ax_xi.errorbar(M_UV_eiger[mask_Hb_SN], xi_ion_eiger[mask_Hb_SN],
            ls='',
            fmt='o', c='dimgray', label=r'S/N(H$\beta$) $>$ 3 EIGER', zorder=-2, rasterized=True)

ax_xi.errorbar(M_UV[mask_COLA1][0], xi_ion_COLA1_percs[1],
            yerr=np.abs([xi_ion_COLA1_percs[0] - xi_ion_COLA1_percs[1],
                  xi_ion_COLA1_percs[2] - xi_ion_COLA1_percs[1]]).reshape(2, 1),
            xerr=np.abs([M_UV_err_down[mask_COLA1][0], M_UV_err_up[mask_COLA1][0]]).reshape(2, 1),
            ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label=r'COLA1', zorder=99999)

print(f'COLA1: {xi_ion_COLA1_percs[1]:0.2f} +/- {-xi_ion_COLA1_percs[1] + xi_ion_COLA1_percs}')

mask_Hb_SN = (cat['f_Hb'] / cat['f_Hb_err'] > 3)
M_UV_median = np.nanmedian(M_UV[mask_Hb_SN])
xi_ion_median = np.nanmedian(xi_ion[mask_Hb_SN])

print(f'All sources: xi_ion = {xi_ion_median:0.2f} +/- {np.abs(xi_ion_median - np.nanpercentile(xi_ion, [16, 84]))}')
ax_xi.errorbar(M_UV_median, xi_ion_median,
            yerr=np.abs(xi_ion_median - np.nanpercentile(xi_ion[mask_Hb_SN], [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median - np.nanpercentile(M_UV[mask_Hb_SN], [16, 84])).reshape(2, 1),
            zorder=999, fmt='D', ms=10, capsize=5, mfc='lightsteelblue', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average C1F')

mask_Hb_SN = (cat_eiger['f_Hb'] / cat_eiger['f_Hb_err'] > 3)
M_UV_median_eiger = np.nanmedian(M_UV_eiger[mask_Hb_SN])
xi_ion_median_eiger = np.nanmedian(xi_ion_eiger[mask_Hb_SN])
print(f'{xi_ion_median_eiger=}')
ax_xi.errorbar(M_UV_median_eiger, xi_ion_median_eiger,
            yerr=np.abs(xi_ion_median_eiger - np.nanpercentile(xi_ion_eiger[mask_Hb_SN], [16, 84])).reshape(2, 1),
            xerr=np.abs(M_UV_median_eiger - np.nanpercentile(M_UV_eiger[mask_Hb_SN], [16, 84])).reshape(2, 1),
            zorder=999, fmt='s', ms=10, capsize=5, mfc='dimgray', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average EIGER')

### Compute median and percentiles in bins of M_UV ###
mask_Hb_SN = cat['f_Hb'] / cat['f_Hb_err'] > 3
mask_Hb_SN_eiger = cat_eiger['f_Hb'] / cat_eiger['f_Hb_err'] > 3
xi_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([xi_ion[mask_Hb_SN], xi_ion_eiger[mask_Hb_SN_eiger]]),
                                                    np.concatenate([M_UV[mask_Hb_SN], M_UV_eiger[mask_Hb_SN_eiger]]),
                                                    -25, -15, 1.4, 0.1)
ax_xi.plot(M_UV_bin_centers, xi_percs[1], c='k', lw=3)
ax_xi.plot(M_UV_bin_centers, xi_percs[1], c='darkcyan', lw=2,
             label='Median (C1F + EIGER)')
ax_xi.fill_between(M_UV_bin_centers, xi_percs[0], xi_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######

ax_xi.set_ylim(24.25, 27)
ax_xi.set_xlim(-22.5, -16.5)

ax_xi.set_xlabel(r'$M_{\rm UV}$')
ax_xi.set_ylabel(r'$\log_{10}(\xi_{\rm ion, 0} / {\rm Hz}\,{\rm erg}^{-1})$')

ax_xi.legend(fontsize=10, markerfirst=False)

ax_xi.tick_params(direction='in', which='both')
ax_xi.xaxis.set_ticks_position('both')
ax_xi.yaxis.set_ticks_position('both')
ax_xi.invert_xaxis()

fig.subplots_adjust(hspace=0.02)

savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/beta_EW_xiion.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
np.abs(EW_median - np.nanpercentile(EW[mask_good_EW], [84, 16])).reshape(2, 1),

In [ ]:
HgHb = 0.4

tau = np.log(0.473 / HgHb)

thisEBV = 0.95 * tau

dustcorr_Hb = 10**(0.4*(thisEBV)*3.76)
dustcorr_UV = 10**(0.4*(thisEBV)*8.68)

print(dustcorr_Hb, dustcorr_UV)

In [ ]:
fig, axes = plt.subplots(3, figsize=(6, 14), sharex=True)

ax_beta = axes[0]
ax_ew = axes[1]
ax_xi = axes[2]

ax_beta.errorbar(M_UV, cat['beta_powerlaw'],
            ls='', fmt='o', c='lightsteelblue', label='Powerlaw', zorder=-1, rasterized=True)

ax_beta.errorbar(cat['muv'], cat['beta'],
            ls='', fmt='o', c='k', label='Prospector', zorder=-1, rasterized=True)


ax_beta.errorbar(cat['muv'][mask_COLA1], cat['beta'][mask_COLA1],
            zorder=999, fmt='D', ms=10, capsize=5, mfc='r', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1')

# https://ui.adsabs.harvard.edu/search/q=%20%20author%3A%22%5EBouwens%22%20year%3A2014&sort=date%20desc%2C%20bibcode%20desc&p_=0
# bouwens 2014 793 apj
xx = np.arange(-23, -18.8, 0.01)
xx_2 = np.arange(-18.8, -16, 0.01)
yy = -2.22 + -0.24*(xx - -18.8)
yy_2 = -2.22 + -0.08*(xx_2 - -18.8)

ax_beta.plot(xx, yy, color='indigo', ls='--', lw=3,
        label=r'LBGs $z\sim6$ (Bouwens+14)')
ax_beta.plot(xx_2, yy_2, color='indigo', ls='--', lw=3)

print(f'COLA1: beta = {cat['beta_powerlaw'][mask_COLA1][0]      :0.2} +/- {cat['beta_powerlaw_err'][mask_COLA1][0]:0.2f}')
print(f'All: beta = {np.nanmedian(cat['beta_powerlaw']):0.2} +/- {
      np.abs(np.nanmedian(cat['beta_powerlaw']) - np.nanpercentile(cat['beta_powerlaw'], [84, 16]))}')

### Compute median and percentiles in bins of M_UV ###
beta_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([cat['beta']]),
                                                    np.concatenate([M_UV]),
                                                    -25, -15, 1.6, 0.1)
ax_beta.plot(M_UV_bin_centers, beta_percs[1], c='k', lw=3)
ax_beta.plot(M_UV_bin_centers, beta_percs[1], c='darkcyan', lw=2,
             label='Median (prospector))')
ax_beta.fill_between(M_UV_bin_centers, beta_percs[0], beta_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######


# ax_beta.set_xlabel(r'$M_{\rm UV}$')
ax_beta.set_ylabel(r'$\beta_{\rm UV}$')

ax_beta.legend(fontsize=10, markerfirst=False)

ax_beta.set_xlim(-22.5, -16.5)
ax_beta.set_ylim(-4.5, 2.5)

ax_beta.tick_params(direction='in', which='both')
ax_beta.xaxis.set_ticks_position('both')
ax_beta.yaxis.set_ticks_position('both')
ax_beta.invert_xaxis()


#################
#################

ax_ew.errorbar(M_UV, EW,
               ls='',
               fmt='o', c='lightsteelblue', label='Powerlaw', zorder=-1, rasterized=True)
ax_ew.errorbar(cat['muv'], cat['EW_Hb'] + cat['EW_O3'],
               ls='',
               fmt='o', c='k', label='Prospector', zorder=-1, rasterized=True)

ax_ew.errorbar(cat['muv'][mask_COLA1], (cat['EW_Hb'] + cat['EW_O3'])[mask_COLA1],
            #    yerr=np.asarray([EW_COLA1_percs[2] - EW_COLA1_percs[0],
            #                    EW_COLA1_percs[1] - EW_COLA1_percs[0]]).reshape(2, 1),
            #    xerr=np.asarray([M_UV_err_up[mask_COLA1][0], M_UV_err_down[mask_COLA1][0]]).reshape(2, 1),
               ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
               elinewidth=2, capthick=2, mew=1.5, label=r'COLA1', zorder=99999)


### Compute median and percentiles in bins of M_UV ###
ew_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([cat['EW_Hb'] + cat['EW_O3']]),
                                                    np.concatenate([cat['muv']]),
                                                    -25, -15, 1.7, 0.1)
ax_ew.plot(M_UV_bin_centers, ew_percs[1], c='k', lw=3)
ax_ew.plot(M_UV_bin_centers, ew_percs[1], c='darkcyan', lw=2,
             label='Median (Prospector)')
ax_ew.fill_between(M_UV_bin_centers, ew_percs[0], ew_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######

ax_ew.set_ylim(12, 5e4)
ax_ew.set_xlim(-22.5, -16.5)

# ax_ew.set_xlabel(r'$M_{\rm UV}$')
ax_ew.set_ylabel(r'EW$_0$([OIII] + H$\beta$) [\AA]')

ax_ew.set_yscale('log')

ax_ew.legend(fontsize=10, markerfirst=False)

ax_ew.tick_params(direction='in', which='both')
ax_ew.xaxis.set_ticks_position('both')
ax_ew.yaxis.set_ticks_position('both')
ax_ew.invert_xaxis()

##################
##################

mask_Hb_SN = cat['f_Hb'] / cat['f_Hb_err'] > 3
ax_xi.errorbar(M_UV[mask_Hb_SN], xi_ion[mask_Hb_SN],
            ls='',
            fmt='o', c='lightsteelblue', label='Powerlaw', zorder=-1, rasterized=True)
ax_xi.errorbar(cat['muv'][mask_Hb_SN], cat['xi_ion_Hb'][mask_Hb_SN],
            ls='',
            fmt='o', c='k', label='Prospector', zorder=-1, rasterized=True)

ax_xi.errorbar(cat['muv'][mask_COLA1], cat['xi_ion_Hb'][mask_COLA1],
            ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label=r'COLA1', zorder=99999)


### Compute median and percentiles in bins of M_UV ###
mask_Hb_SN = cat['f_Hb'] / cat['f_Hb_err'] > 3
mask_Hb_SN_eiger = cat_eiger['f_Hb'] / cat_eiger['f_Hb_err'] > 3
xi_percs, M_UV_bin_centers = M_UV_bin_percentiles(np.concatenate([cat['xi_ion_Hb'][mask_Hb_SN]]),
                                                    np.concatenate([cat['muv'][mask_Hb_SN]]),
                                                    -25, -15, 1.4, 0.1)
ax_xi.plot(M_UV_bin_centers, xi_percs[1], c='k', lw=3)
ax_xi.plot(M_UV_bin_centers, xi_percs[1], c='darkcyan', lw=2,
             label='Median (Prospector)')
ax_xi.fill_between(M_UV_bin_centers, xi_percs[0], xi_percs[2],
                color='darkcyan', lw=0, alpha=0.35)
######

ax_xi.set_ylim(24, 27)
ax_xi.set_xlim(-22.5, -16.5)

ax_xi.set_xlabel(r'$M_{\rm UV}$')
ax_xi.set_ylabel(r'$\log_{10}(\xi_{\rm ion, 0} / {\rm Hz}\,{\rm erg}^{-1})$')

ax_xi.legend(fontsize=10, markerfirst=False)

ax_xi.tick_params(direction='in', which='both')
ax_xi.xaxis.set_ticks_position('both')
ax_xi.yaxis.set_ticks_position('both')
ax_xi.invert_xaxis()

fig.subplots_adjust(hspace=0.02)

savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
# fig.savefig(f'{savefig_path}/beta_EW_xiion.pdf', bbox_inches='tight', pad_inches=0.1,
#             facecolor='w')
plt.show()

In [ ]:
mask_Hb_SN = cat['f_Hb'] / cat['f_Hb_err'] > 3
sum(mask_Hb_SN)